In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from collections import Counter
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
# Model Building
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
#--------------------------------------------

#Goal: create the final dataset from the 7 email phishing dataset (the union of all small datasets)
# list of current baby sets: data1, data2, data3, data4, data5, data6, data7
# to do list:
# 1. try to reach a good number of emails, around 100k        [  X  ] 
# 2. figure out what to do with the Nan datasets: data2_nan, data3_nan,data4_nan,data5_nan  [  ]
# 3. deal witht he weird ascii data in all datasets, best to remove them imo, but for later [  ]
# 4. before joining the small datasets, make sure to remove sender from dataset 1, 2 , 3 , 4, 6. and add url (1 if contains, 0 if not) to dataset 5, 7. [ ] 
# statistics:
# phishing emails: 94,305    / Safe emails: 99,267    / Total emails: 202,918 (numbers dont add to that exacttly, probably some nan values or other that need fixing or some error..look into it.

#--------------------------------------------
data1 = pd.read_csv("../datasets/CEAS_08.csv",encoding='latin1')
#CEAS 8 Dataset, has after removing the unwanted columns (39154, 5),   21842 is phishing, 17312 is safe , dtype of label is int64

#preprocessing so far for it ( no vectorization, just data cleaning...etc)
data1.drop(columns=["receiver", "date"], inplace=True)
data1.rename(columns={'label': 'isPhishing'}, inplace=True) 

#--------------------------------------------

data2 = pd.read_csv("./datasets/TREC_07.csv", encoding='latin1', engine='python', on_bad_lines='skip')
#TREC 7 Dataset, has (53745, 5),   29392 is phishing, 24353   is safe , dtype of label is now int64, used to be object with other out of place labels( None, '   ', 'const')
data2.drop(columns=["receiver", "date"], inplace=True)
data2.rename(columns={'label': 'isPhishing'}, inplace=True) 

#could probably do this step in a ..cleaner more effecient way...removing any value other than 1 or 0
data2_placeholder= data2[data2['isPhishing'].isin(['0', '1'])]
data2 = data2_placeholder
del data2_placeholder

#i removed the rows with label Nan in "isPhishing", will see what to do with that later, saved the Nan rows to data2_nan
data2_nan = data2[data2['isPhishing'].isna()]
data2 = data2.dropna(subset=['isPhishing'])

#--------------------------------------------

data3 = pd.read_csv("./datasets/TREC_06.csv", encoding='latin1',index_col=0, engine='python', on_bad_lines='skip') # i added teh index col because it fixed a problem..god bless stackoverflow

# Trec 6, has (16382, 5), 3989 are phsihing, 12393 are safe, dtype of label is int 64 as well, make sure numbers are updated after cleaning
#NAn problem handled first, decide to kill it or keep it later
data3_nan = data3[data3['label'].isna()]
data3 = data3.dropna(subset=['label'])
data3.drop(columns=["receiver", "date"], inplace=True)
data3.rename(columns={'label': 'isPhishing'}, inplace=True) 

#--------------------------------------------

data4 = pd.read_csv("./datasets/TREC_05.csv", encoding='latin1', engine='python', on_bad_lines='skip')
#Trec 5, has (55210, 5), 22932 are phishing, 22932 are safe
#Nan Problem
data4_nan = data4[data4['label'].isna()]
data4 = data4.dropna(subset=['label'])
#Label mismatch type problem 
data4 = data4[data4['label'].isin(['0', '1'])]
data4['label'] = data4['label'].astype(int)

#dropping 2 columns and renaming one
data4.drop(columns=["receiver", "date"], inplace=True)
data4.rename(columns={'label': 'isPhishing'}, inplace=True) 

#--------------------------------------------
data5 = pd.read_csv("./datasets/Enron.csv", encoding='latin1', engine='python', on_bad_lines='skip')
#features: subject, body, isphishing
#Enron , has (29763, 3), 13976 are phishing, 15787 safe, Enron has weird emails probably better to discard it!!!!!!!!!!!!!!!!!!!!
data5.rename(columns={'label': 'isPhishing'}, inplace=True)
data5_nan = data5[data5['isPhishing'].isna()]
data5 = data5.dropna(subset=['isPhishing'])
data5 = data5[data5['isPhishing'].isin(['0', '1'])]
data5['isPhishing'] = data5['isPhishing'].astype('int64')

#--------------------------------------------
data6 = pd.read_csv("./datasets/Assassin.csv", encoding='latin1', engine='python', on_bad_lines='skip')
#Assassin dataset, (5805, 5), 1716 phishing, 4089 safe, 
data6.drop(columns=["receiver", "date"], inplace=True)
data6.rename(columns={'label': 'isPhishing'}, inplace=True) 
data6_nan = data6[data6['isPhishing'].isna()]
data6 = data6.dropna(subset=['isPhishing'])
data6['isPhishing'] = data6['isPhishing'].round().astype('int64')

#--------------------------------------------
data7 = pd.read_csv("./datasets/Ling.csv", encoding='latin1', engine='python', on_bad_lines='skip')
# Ling, has 3 features subject, body, isPhishing,(2859, 3), 458 phishing emails, 2401 safe emails.
data7.rename(columns={'label': 'isPhishing'}, inplace=True) 
data7_nan = data7[data7['isPhishing'].isna()]
data7 = data7.dropna(subset=['isPhishing'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omark\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omark\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# some useful functions!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#to check a specific row
data1.loc[21609, 'body']
#check a sample
data1.sample(15)
#check features/ columns
data1.columns
#check general overview (rows and number of columns)
data1.shape
#see how many rows are in the dataset
len(data1)

# delete a dataset
del data223344
#drop a column
data4.drop(columns=["c1", "c2"], inplace=True)
# rename a column
data4.rename(columns={'oldname': 'newname'}, inplace=True) 
#check type and number of unique values
print(data1['isPhishing'].dtype)     
print(data1['isPhishing'].unique())   # check the unique values returns an array
# check the number of rows with each value
data1['label'].value_counts()
data3['label'].value_counts(dropna=False)

#convert to another datatype
data1['label'] = data['label'].astype(new data type)
#keep certain values:
# Keep only rows where label is 0 or 1
data3 = data3[data3['label'].isin(['0', '1'])]



